In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 60*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240404033920
0	Validation losses: 0.4631, 32.4235, 32.8865	Best Loss: 32.8865 (0)	Accuracy: 82.44%
1	Validation losses: 0.4244, 10.4042, 10.8286	Best Loss: 10.8286 (0)	Accuracy: 85.68%
2	Validation losses: 0.4612, 1.6886, 2.1498	Best Loss: 2.1498 (0)	Accuracy: 83.60%
3	Validation losses: 0.5047, 0.4313, 0.9360	Best Loss: 0.9360 (0)	Accuracy: 81.88%
4	Validation losses: 0.4757, 0.3297, 0.8055	Best Loss: 0.8055 (0)	Accuracy: 82.90%
5	Validation losses: 0.5350, 0.3079, 0.8429	Best Loss: 0.8055 (1)	Accuracy: 79.66%
6	Validation losses: 0.4484, 0.2869, 0.7353	Best Loss: 0.7353 (0)	Accuracy: 84.60%
7	Validation losses: 0.6142, 0.2495, 0.8636	Best Loss: 0.7353 (1)	Accuracy: 77.80%
8	Validation losses: 0.4981, 0.2077, 0.7058	Best Loss: 0.7058 (0)	Accuracy: 82.54%
9	Validation losses: 0.6190, 0.2213, 0.8402	Best Loss: 0.7058 (1)	Accuracy: 77.98%
10	Validation losses: 0.5840, 0.1980, 0.7820	Best Loss: 0.7058 (2)	Accuracy: 81.14%
11	Validation losses: 0

23	Validation losses: 2.3061, 0.5062, 2.8123	Best Loss: 0.6990 (10)	Accuracy: 9.86%
24	Validation losses: 2.3045, 0.4879, 2.7924	Best Loss: 0.6990 (11)	Accuracy: 9.86%
25	Validation losses: 2.3053, 0.4778, 2.7831	Best Loss: 0.6990 (12)	Accuracy: 9.86%
26	Validation losses: 2.3057, 0.4701, 2.7758	Best Loss: 0.6990 (13)	Accuracy: 9.14%
27	Validation losses: 2.3027, 0.4647, 2.7674	Best Loss: 0.6990 (14)	Accuracy: 9.76%
28	Validation losses: 2.3044, 0.4592, 2.7636	Best Loss: 0.6990 (15)	Accuracy: 9.80%
29	Validation losses: 2.3041, 0.4545, 2.7586	Best Loss: 0.6990 (16)	Accuracy: 10.08%
30	Validation losses: 2.3037, 0.4513, 2.7550	Best Loss: 0.6990 (17)	Accuracy: 10.08%
31	Validation losses: 2.3027, 0.4485, 2.7513	Best Loss: 0.6990 (18)	Accuracy: 11.12%
32	Validation losses: 2.3040, 0.4470, 2.7509	Best Loss: 0.6990 (19)	Accuracy: 9.14%
33	Validation losses: 2.3034, 0.4450, 2.7484	Best Loss: 0.6990 (20)	Accuracy: 10.02%
Early stopping!
Total running time:	 2735
INFO:tensorflow:Restoring para

3	Validation losses: 0.4639, 0.9144, 1.3784	Best Loss: 1.3784 (0)	Accuracy: 82.76%
4	Validation losses: 0.4686, 0.3415, 0.8101	Best Loss: 0.8101 (0)	Accuracy: 82.84%
5	Validation losses: 0.4903, 0.2708, 0.7611	Best Loss: 0.7611 (0)	Accuracy: 82.06%
6	Validation losses: 0.4652, 0.2515, 0.7167	Best Loss: 0.7167 (0)	Accuracy: 83.34%
7	Validation losses: 0.4882, 0.2430, 0.7313	Best Loss: 0.7167 (1)	Accuracy: 82.20%
8	Validation losses: 0.4824, 0.2296, 0.7120	Best Loss: 0.7120 (0)	Accuracy: 83.30%
9	Validation losses: 1.2079, 0.2064, 1.4143	Best Loss: 0.7120 (1)	Accuracy: 67.56%
10	Validation losses: 0.4826, 0.1757, 0.6583	Best Loss: 0.6583 (0)	Accuracy: 83.14%
11	Validation losses: 0.4785, 0.1638, 0.6423	Best Loss: 0.6423 (0)	Accuracy: 83.62%
12	Validation losses: 0.5612, 0.1420, 0.7032	Best Loss: 0.6423 (1)	Accuracy: 81.34%
13	Validation losses: 0.4823, 0.1241, 0.6064	Best Loss: 0.6064 (0)	Accuracy: 83.26%
14	Validation losses: 0.4641, 0.1360, 0.6000	Best Loss: 0.6000 (0)	Accuracy: 84.18%

Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    14.2300009727%
RANDOM SEED:   2 	LAMBDA_EXP:   -4.4
Now: 20240404094500
0	Validation losses: 0.4042, 17.0833, 17.4875	Best Loss: 17.4875 (0)	Accuracy: 85.44%
1	Validation losses: 0.3773, 11.3549, 11.7322	Best Loss: 11.7322 (0)	Accuracy: 86.24%
2	Validation losses: 0.3990, 5.7164, 6.1154	Best Loss: 6.1154 (0)	Accuracy: 85.82%
3	Validation losses: 0.5191, 2.1559, 2.6750	Best Loss: 2.6750 (0)	Accuracy: 81.00%
4	Validation losses: 0.3928, 0.7098, 1.1027	Best Loss: 1.1027 (0)	Accuracy: 85.84%
5	Validation losses: 0.4439, 0.3138, 0.7577	Best Loss: 0.7577 (0)	Accuracy: 83.42%
6	Validation losses: 0.4529, 0.2330, 0.6859	Best Loss: 0.6859 (0)	Accuracy: 83.64%
7	Validation losses: 0.4456, 0.2152, 0.6608	Best Loss: 0.6608 (0)	Accuracy: 84.52%
8	Validation losses: 0.4024, 0.2059, 0.6083	Best Loss: 0.6083 (0)	Accuracy: 85.02%
9	Validation losses: 0.4417, 0

30	Validation losses: 0.4162, 0.0869, 0.5031	Best Loss: 0.4886 (2)	Accuracy: 86.14%
31	Validation losses: 0.4022, 0.0866, 0.4888	Best Loss: 0.4886 (3)	Accuracy: 85.60%
32	Validation losses: 0.3892, 0.0844, 0.4736	Best Loss: 0.4736 (0)	Accuracy: 86.44%
33	Validation losses: 0.3955, 0.0840, 0.4795	Best Loss: 0.4736 (1)	Accuracy: 86.32%
34	Validation losses: 0.3878, 0.0833, 0.4711	Best Loss: 0.4711 (0)	Accuracy: 86.90%
35	Validation losses: 0.3824, 0.0821, 0.4645	Best Loss: 0.4645 (0)	Accuracy: 86.78%
36	Validation losses: 0.3796, 0.0812, 0.4608	Best Loss: 0.4608 (0)	Accuracy: 86.24%
37	Validation losses: 0.3678, 0.0806, 0.4485	Best Loss: 0.4485 (0)	Accuracy: 87.12%
38	Validation losses: 0.3726, 0.0803, 0.4529	Best Loss: 0.4485 (1)	Accuracy: 86.80%
39	Validation losses: 0.3596, 0.0796, 0.4392	Best Loss: 0.4392 (0)	Accuracy: 87.44%
40	Validation losses: 0.3588, 0.0796, 0.4384	Best Loss: 0.4384 (0)	Accuracy: 87.38%
41	Validation losses: 0.3609, 0.0797, 0.4406	Best Loss: 0.4384 (1)	Accuracy:

...

14	Validation losses: 0.3106, 0.2613, 0.5719	Best Loss: 0.5719 (0)	Accuracy: 88.46%
15	Validation losses: 0.3130, 0.2549, 0.5679	Best Loss: 0.5679 (0)	Accuracy: 88.74%
16	Validation losses: 0.3314, 0.2488, 0.5802	Best Loss: 0.5679 (1)	Accuracy: 88.38%
17	Validation losses: 0.2961, 0.2426, 0.5387	Best Loss: 0.5387 (0)	Accuracy: 89.84%
18	Validation losses: 0.3299, 0.2370, 0.5669	Best Loss: 0.5387 (1)	Accuracy: 88.46%
19	Validation losses: 0.3548, 0.2313, 0.5861	Best Loss: 0.5387 (2)	Accuracy: 87.98%
20	Validation losses: 0.3659, 0.2258, 0.5917	Best Loss: 0.5387 (3)	Accuracy: 87.56%
21	Validation losses: 0.3346, 0.2197, 0.5543	Best Loss: 0.5387 (4)	Accuracy: 88.78%
22	Validation losses: 0.3912, 0.2144, 0.6056	Best Loss: 0.5387 (5)	Accuracy: 88.44%
23	Validation losses: 0.3246, 0.2083, 0.5329	Best Loss: 0.5329 (0)	Accuracy: 89.32%
24	Validation losses: 0.3116, 0.2026, 0.5142	Best Loss: 0.5142 (0)	Accuracy: 89.86%
25	Validation losses: 0.3813, 0.1975, 0.5788	Best Loss: 0.5142 (1)	Accuracy:

20	Validation losses: 0.3347, 0.1793, 0.5140	Best Loss: 0.4968 (14)	Accuracy: 88.42%
21	Validation losses: 0.3244, 0.1775, 0.5019	Best Loss: 0.4968 (15)	Accuracy: 89.28%
22	Validation losses: 0.3275, 0.1754, 0.5029	Best Loss: 0.4968 (16)	Accuracy: 89.66%
23	Validation losses: 0.3604, 0.1733, 0.5336	Best Loss: 0.4968 (17)	Accuracy: 89.62%
24	Validation losses: 0.3326, 0.1711, 0.5037	Best Loss: 0.4968 (18)	Accuracy: 89.80%
25	Validation losses: 0.3187, 0.1689, 0.4877	Best Loss: 0.4877 (0)	Accuracy: 90.40%
26	Validation losses: 0.3415, 0.1666, 0.5081	Best Loss: 0.4877 (1)	Accuracy: 90.06%
27	Validation losses: 0.3680, 0.1644, 0.5324	Best Loss: 0.4877 (2)	Accuracy: 89.94%
28	Validation losses: 0.3739, 0.1622, 0.5361	Best Loss: 0.4877 (3)	Accuracy: 90.24%
29	Validation losses: 0.3754, 0.1603, 0.5357	Best Loss: 0.4877 (4)	Accuracy: 89.94%
30	Validation losses: 0.4090, 0.1584, 0.5674	Best Loss: 0.4877 (5)	Accuracy: 89.58%
31	Validation losses: 0.4354, 0.1566, 0.5920	Best Loss: 0.4877 (6)	Accu

Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.6799998283%
RANDOM SEED:   2 	LAMBDA_EXP:   -6.6
Now: 20240406011842
0	Validation losses: 0.4095, 0.1283, 0.5378	Best Loss: 0.5378 (0)	Accuracy: 84.86%
1	Validation losses: 0.3604, 0.1283, 0.4887	Best Loss: 0.4887 (0)	Accuracy: 87.04%
2	Validation losses: 0.3472, 0.1283, 0.4755	Best Loss: 0.4755 (0)	Accuracy: 87.40%
3	Validation losses: 0.3483, 0.1283, 0.4766	Best Loss: 0.4755 (1)	Accuracy: 86.58%
4	Validation losses: 0.3319, 0.1283, 0.4602	Best Loss: 0.4602 (0)	Accuracy: 88.50%
5	Validation losses: 0.3411, 0.1283, 0.4694	Best Loss: 0.4602 (1)	Accuracy: 87.98%
6	Validation losses: 0.3121, 0.1283, 0.4404	Best Loss: 0.4404 (0)	Accuracy: 88.70%
7	Validation losses: 0.3130, 0.1283, 0.4413	Best Loss: 0.4404 (1)	Accuracy: 88.90%
8	Validation losses: 0.3128, 0.1283, 0.4412	Best Loss: 0.4404 (2)	Accuracy: 88.70%
9	Validation losses: 0.3676, 0.1284, 0.4961	Best Loss: 0.4404

26	Validation losses: 0.3539, 0.0919, 0.4458	Best Loss: 0.3829 (15)	Accuracy: 89.82%
27	Validation losses: 0.4076, 0.0918, 0.4994	Best Loss: 0.3829 (16)	Accuracy: 89.40%
28	Validation losses: 0.3824, 0.0916, 0.4740	Best Loss: 0.3829 (17)	Accuracy: 90.48%
29	Validation losses: 0.3752, 0.0914, 0.4666	Best Loss: 0.3829 (18)	Accuracy: 90.14%
30	Validation losses: 0.4061, 0.0912, 0.4973	Best Loss: 0.3829 (19)	Accuracy: 90.16%
31	Validation losses: 0.4424, 0.0910, 0.5334	Best Loss: 0.3829 (20)	Accuracy: 90.42%
Early stopping!
Total running time:	 2648
INFO:tensorflow:Restoring parameters from ./models/model_20240406024512.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.4900016785%
RANDOM SEED:   2 	LAMBDA_EXP:   -6.8
Now: 20240406032921
0	Validation losses: 0.3988, 0.0810, 0.4798	Best Loss: 0.4798 (0)	Accuracy: 85.82%
1	Validation losses: 0.3554, 0.0811, 0.4364	Best Loss: 0.4364 (0)	Accur

22	Validation losses: 0.3126, 0.0614, 0.3741	Best Loss: 0.3563 (14)	Accuracy: 90.12%
23	Validation losses: 0.3147, 0.0618, 0.3765	Best Loss: 0.3563 (15)	Accuracy: 90.18%
24	Validation losses: 0.3879, 0.0621, 0.4500	Best Loss: 0.3563 (16)	Accuracy: 88.88%
25	Validation losses: 0.3570, 0.0623, 0.4193	Best Loss: 0.3563 (17)	Accuracy: 89.16%
26	Validation losses: 0.3595, 0.0625, 0.4220	Best Loss: 0.3563 (18)	Accuracy: 89.80%
27	Validation losses: 0.3831, 0.0626, 0.4457	Best Loss: 0.3563 (19)	Accuracy: 89.86%
28	Validation losses: 0.3589, 0.0626, 0.4215	Best Loss: 0.3563 (20)	Accuracy: 90.50%
Early stopping!
Total running time:	 2394
INFO:tensorflow:Restoring parameters from ./models/model_20240406044929.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    12.2299993038%
RANDOM SEED:   2 	LAMBDA_EXP:   -7.0
Now: 20240406052924
0	Validation losses: 0.3964, 0.0511, 0.4475	Best Loss: 0.4475 (0)	Acc